<a href="https://colab.research.google.com/github/DGuilherme/TurbofanVibration/blob/master/TurbofanVibrationDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analise do Data Set Turbofans
